# DATA SUMMARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn import cluster
import sklearn
import seaborn as sns
from sklearn import decomposition
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from scipy.stats import boxcox
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [13]:
# allow us to see all columns
pd.set_option('display.max_columns', None)

In [31]:
df = pd.read_csv("../Datasets_P1/df_filled_important.csv")
df

C:\Users\lukep\AppData\Local\Temp\ipykernel_19988\2335678479.py:1: DtypeWarning: Columns (8,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Datasets_P1/df_filled_important.csv")


,rpt_rec_num,Provider_CCN,Facility_Name,Street_Address,City,State_Code,Zip_Code,County,Medicare_CBSA_Number,Rural_versus_Urban,Fiscal_Year_Begin_Date,Fiscal_Year_End_Date,Type_of_Control,Accounts_Receivable,Accounts_payable,Buildings,Cash_on_hand_and_in_banks,General_fund_balance,Gross_Revenue,Inpatient_PPS_Amount,Inpatient_Revenue,Less_Total_Operating_Expense,Less_discounts_on_patients,Major_movable_equipment,Net_Income,Net_Patient_Revenue,Number_of_Beds,Other_Assets,Other_current_liabilities,Overhead_Non_Salary_Costs,SNF_Admissions_Other,SNF_Admissions_Title_XIX,SNF_Admissions_Title_XVIII,SNF_Admissions_Total,SNF_Days_Other,SNF_Days_Title_XIX,SNF_Days_Title_XVIII,SNF_Days_Total,SNF_Discharges_Title_Other,SNF_Discharges_Title_XIX,SNF_Discharges_Title_XVIII,SNF_Discharges_Total,Salaries_wages_and_fees_payable,Total_Assets,Total_Bed_Days_Available,Total_Costs,Total_Days_Other,Total_Days_Title_XIX,Total_Days_Title_XVIII,Total_Days_Total,Total_Discharges_Title_Other,Total_Discharges_Title_XIX,Total_Discharges_Title_XVIII,Total_Discharges_Total,Total_Income,Total_RUG_Days,Total_current_liabilities,Total_fixed_Assets,Total_fund_balances,Total_liabilities,Total_other_Assets,Fixed_equipment,Total_Charges,SNF_Number_of_Beds,Total_long_term_liabilities,Investments,Net_Income_from_service_to_patients,Wage-related_Costs_(core),Total_Salaries_(adjusted),SNF_Average_Length_of_Stay_Total,Notes_Payable,Other_long_term_liabilities,Allowable_Bad_Debts,SNF_Average_Length_of_Stay_Title_XVIII,Total_General_Inpatient_Care_Services_Revenue,Total_Liabilities_and_fund_balances,SNF_Average_Length_of_Stay_Title_XIX,Total_Current_Assets,Total_Other_Income,SNF_Bed_Days_Available,Land,Prepaid_expenses,Year
0,1089712,495134,RIDGECREST MANOR NURSING & REHAB CTR,157 ROSS-CARTER BLVD,DUFFIELD,VA,24244,SCOTT,28700.0,U,11/1/2014,12/31/2014,4.0,1366877.0,292277.0,NaN,54091.0,311430.0,1800296.0,154022.0,1800296.0,1209322.0,279666.0,59343.0,311430.0,1520630.0,120.0,NaN,5667.0,549090.0,2.0,40.0,6.0,48.0,534.0,5853.0,333.0,6720.0,9.0,35.0,5.0,49.0,117929.0,765020.0,7320.0,211585.0,534.0,5853.0,333.0,6720.0,9.0,35.0,5.0,49.0,311431.0,333.0,453591.0,55061.0,311430.0,453590.0,-729268.0,NaN,NaN,120.0,-1.0,NaN,311308.0,80664.0,660232.0,137.14,NaN,NaN,NaN,66.60,1182049.0,765020.0,167.23,1439227.0,123.0,NaN,NaN,NaN,2015
1,1091410,75417,REGENCY HEIGHTS OF NORWICH LLC,60 CROUCH AVENUE,NORWICH,CT,6360,NEW LONDON,35980.0,U,10/1/2014,12/14/2014,4.0,1326379.0,NaN,NaN,-339775.0,-4896947.0,2843541.0,393088.0,2843541.0,3005301.0,426716.0,407352.0,-588469.0,2416825.0,114.0,36000.0,2596536.0,1886441.0,38.0,NaN,23.0,61.0,7241.0,NaN,856.0,8097.0,36.0,NaN,23.0,59.0,12590.0,1687456.0,8550.0,207903.0,7241.0,NaN,856.0,8097.0,36.0,NaN,23.0,59.0,-588469.0,856.0,2765768.0,671306.0,-4896947.0,6584403.0,36000.0,NaN,NaN,114.0,3818635.0,NaN,-588476.0,559430.0,1118860.0,137.24,NaN,NaN,NaN,37.22,2429100.0,1687456.0,NaN,980150.0,7.0,NaN,NaN,NaN,2015
2,1093283,165252,WESTVIEW ACRES CARE CENTER,203 SW LORRAINE,LEON,IA,50144,DECATUR,99916.0,R,10/1/2014,12/31/2014,4.0,135859.0,79894.0,NaN,87279.0,52141.0,613243.0,20234.0,613243.0,639168.0,13210.0,NaN,-47859.0,600033.0,60.0,NaN,25982.0,290554.0,5.0,4.0,3.0,12.0,956.0,2548.0,71.0,3575.0,3.0,3.0,3.0,9.0,48546.0,244239.0,5520.0,22920.0,956.0,2548.0,71.0,3575.0,3.0,3.0,3.0,9.0,-30743.0,71.0,192098.0,NaN,52141.0,192098.0,NaN,NaN,NaN,60.0,0.0,NaN,-39135.0,42209.0,348614.0,397.22,NaN,NaN,NaN,23.67,596683.0,244239.0,849.33,244239.0,8392.0,NaN,NaN,NaN,2015
3,1095547,225497,VERO HEALTH & REHAB PARKWAY,1190 VFW PARKWAY,WEST ROXBURY,MA,2132,SUFFOLK,14454.0,U,11/1/2014,12/31/2014,4.0,1222903.0,537451.0,NaN,352766.0,6382.0,1935277.0,172528.0,1935277.0,1444469.0,557114.0,NaN,-63638.0,1378163.0,141.0,NaN,97267.0,839217.0,15.0,15.0,9.0,39.0,2883.0,2881.0,349.0,6113.0,13.0,13.0,6.0,32.0,201598.0,1830758.0,8601.0,158386.0,2883.0,2881.0,349.0,6113.0,13.0,13.0,6.0,32.0,-63638.0,349.0,1824376.0,26949.0,6382.0,1824376.0,74985.0,NaN,NaN,141.0,0.0,NaN,-66306.0,113676.0,601118.0,

In [32]:
from sklearn.impute import SimpleImputer

numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Step 2: Impute missing values for numeric columns
numeric_imputer = SimpleImputer(strategy='constant', fill_value=0)
df[numeric_columns] = numeric_imputer.fit_transform(df[numeric_columns])
    
# Step 3: Impute missing values for categorical columns
categorical_imputer = SimpleImputer(strategy='constant', fill_value='Unknown')
df[categorical_columns] = categorical_imputer.fit_transform(df[categorical_columns])

# Step 4: Convert appropriate columns to correct data types
df['Fiscal_Year_Begin_Date'] = pd.to_datetime(df['Fiscal_Year_Begin_Date'], errors='coerce')
df['Fiscal_Year_End_Date'] = pd.to_datetime(df['Fiscal_Year_End_Date'], errors='coerce')

# Step 5: Remove columns with a lot of missing Data... basing off of these columns. 
df = df[~((df['Number_of_Beds'].isna()) | (df['Number_of_Beds'] == 0) | 
          (df['Medicare_CBSA_Number'].isna()) | (df['Medicare_CBSA_Number'] == "Unknown"))]


In [33]:
df.describe()

,rpt_rec_num,Provider_CCN,Fiscal_Year_Begin_Date,Fiscal_Year_End_Date,Type_of_Control,Accounts_Receivable,Accounts_payable,Buildings,Cash_on_hand_and_in_banks,General_fund_balance,Gross_Revenue,Inpatient_PPS_Amount,Inpatient_Revenue,Less_Total_Operating_Expense,Less_discounts_on_patients,Major_movable_equipment,Net_Income,Net_Patient_Revenue,Number_of_Beds,Other_Assets,Other_current_liabilities,Overhead_Non_Salary_Costs,SNF_Admissions_Other,SNF_Admissions_Title_XIX,SNF_Admissions_Title_XVIII,SNF_Admissions_Total,SNF_Days_Other,SNF_Days_Title_XIX,SNF_Days_Title_XVIII,SNF_Days_Total,SNF_Discharges_Title_Other,SNF_Discharges_Title_XIX,SNF_Discharges_Title_XVIII,SNF_Discharges_Total,Salaries_wages_and_fees_payable,Total_Assets,Total_Bed_Days_Available,Total_Costs,Total_Days_Other,Total_Days_Title_XIX,Total_Days_Title_XVIII,Total_Days_Total,Total_Discharges_Title_Other,Total_Discharges_Title_XIX,Total_Discharges_Title_XVIII,Total_Discharges_Total,Total_Income,Total_RUG_Days,Total_current_liabilities,Total_fixed_Assets,Total_fund_balances,Total_liabilities,Total_other_Assets,Fixed_equipment,Total_Charges,SNF_Number_of_Beds,Total_long_term_liabilities,Investments,Net_Income_from_service_to_patients,Wage-related_Costs_(core),Total_Salaries_(adjusted),SNF_Average_Length_of_Stay_Total,Notes_Payable,Other_long_term_liabilities,Allowable_Bad_Debts,SNF_Average_Length_of_Stay_Title_XVIII,Total_General_Inpatient_Care_Services_Revenue,Total_Liabilities_and_fund_balances,SNF_Average_Length_of_Stay_Title_XIX,Total_Current_Assets,Total_Other_Income,SNF_Bed_Days_Available,Prepaid_expenses,Year
count,1.038200e+05,103820.000000,103820,103820,103820.000000,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,103820.000000,103820.000000,103820.000000,103820.000000,103820.000000,1.038200e+05,103820.000000,1.038200e+05,103820.000000,103820.000000,103820.000000,103820.000000,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,103820.000000,1.038200e+05,103820.000000,1.038200e+05,103820.000000,103820.000000,103820.000000,103820.000000,1.038200e+05,103820.000000,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,103820.000000,1.038200e+05,1.038200e+05,1.038200e+05,103820.000000,1.038200e+05,1.038200e+05,103820.000000,1.038200e+05,1.038200e+05,1.038200e+05,1.038200e+05,103820.000000
mean,1.245960e+06,297326.944683,2018-01-27 22:20:06.426507776,2019-01-09 08:31:12.748988416,4.143749,2.235350e+06,9.318466e+05,6.706163e+06,2.545151e+06,3.262579e+06,1.179269e+07,2.070871e+06,1.169685e+07,1.008509e+07,2.504158e+06,9.585754e+05,7.245043e+04,9.288536e+06,2.363766e+02,2.469046e+06,9.913144e+05,5.788647e+06,105.999201,39.905837,122.735407,268.856087,8716.259170,1.689435e+04,3725.999701,2.935571e+04,115.528992,54.789222,112.372086,281.088740,4.727532e+05,1.589561e+07,4.085853e+04,1.306282e+06,10911.871046,1.716427e+04,3760.421778,3.185627e+04,116.502495,55.571085,112.381333,286.987353,2.073168e+05,2611.056983,4.516506e+06,6.162541e+06,3.296818e+06,1.258529e+07,3.632476e+06,1.123397e+06,5.931071e+05,2.267133e+02,8.067484e+06,3.197940e+05,-7.965512e+05,8.261934e+05,4.138256e+06,172.078520,4.333563e+05,1.014020e+06,2.388484e+04,47.902207,9.244467e+06,1.588211e+07,520.668463,6.100594e+06,1.002955e+06,1.085945e+04,4.345970e+04,2017.998671
min,1.089712e+06,15009.000000,2014-10-01 00:00:00,2014-12-14 00:00:00,1.000000,-6.628446e+07,-9.573823e+08,-2.877344e+07,-5.222312e+08,-1.000000e+09,-1.669697e+07,0.000000e+00,-1.669697e+07,0.000000e+00,-1.814712e+07,-3.784215e+06,-1.674696e+08,-6.586137e+07,1.000000e+00,-8.936264e+07,-1.520153e+08,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-8.691037e+06,-1.899618e+08,0.0

In [ ]:
# Total_Days_Total / Total_Bed_Days_Available = Occupancy Rate 

In [34]:
df['Provider_CCN'].nunique()


14941